In [1]:
import pandas as pd
import numpy as np

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import nltk

from gensim.models import Word2Vec

from sklearn.metrics.pairwise import cosine_similarity

from scipy.stats import pearsonr

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/elicer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/elicer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
filtered_music_log_df = pd.read_csv('filtered_user_music_log.csv')
music_info_df = pd.read_csv('all_music_data.csv')
music_total_data = pd.read_csv('music_total.csv')

In [3]:
filtered_music_log_df_renamed = filtered_music_log_df.rename(columns={"TITLE": "Title", "ARTIST": "Artist"})

music_data = filtered_music_log_df_renamed.merge(
    music_info_df[['Title', 'Artist', 'Lyrics']],
    on=['Title', 'Artist'],
    how='left'
)

In [4]:
music_tag_data = pd.merge(
    music_total_data,
    music_info_df[['Title', 'Artist', 'Genre']],
    on=['Title', 'Artist'],
    how='left')

In [5]:
# 무작위로 100명의 사용자 ID를 선택
unique_users = music_data['USER_ID'].unique()

# 해당 사용자의 모든 청취 기록을 가져오기
selected_user_records = music_data[music_data['USER_ID'].isin(unique_users)]

In [6]:
# 영어 불용어 리스트를 로딩
stop_words = set(stopwords.words('english'))

# 가사 전처리 함수 정의
def preprocess_lyrics(lyrics):
    # lyrics가 문자열인지 확인
    if not isinstance(lyrics, str):
        return []  # lyrics가 문자열이 아닐 경우 빈 리스트 반환

    # lyrics가 문자열일 경우 기존의 전처리를 계속 진행
    lyrics = lyrics.lower()
    lyrics = lyrics.translate(str.maketrans('', '', string.punctuation))
    word_tokens = word_tokenize(lyrics)
    filtered_lyrics = [word for word in word_tokens if not word in stop_words]
    return filtered_lyrics

# 전체 데이터셋에 대해 가사 전처리 수행
music_data['Processed_Lyrics'] = music_data['Lyrics'].apply(preprocess_lyrics)

In [7]:
# Word2Vec 모델을 훈련시키기 위한 가사 데이터셋 준비
lyrics_corpus = music_data['Processed_Lyrics'].tolist()

# Word2Vec 모델 초기화 및 훈련
# 차원 수를 100으로 설정하고, 주변 단어(window)를 3개로 설정, 최소 단어 빈도수(min_count)를 2로 설정
w2v_model = Word2Vec(sentences=lyrics_corpus, vector_size=100, window=5, min_count=2, workers=4)

In [8]:
# 모든 가사에서 가장 흔한 단어를 추출하고 가중치를 계산하는 함수를 정의합니다.
def get_top_words_weights(lyrics_list, top_n=20):
    # 모든 가사를 하나의 리스트로 결합합니다.
    all_words = [word for lyrics in lyrics_list for word in lyrics]
    # 가장 흔한 단어와 그 빈도수를 계산합니다.
    top_words = pd.Series(all_words).value_counts().head(top_n)
    # 가중치를 계산합니다: 여기서는 단순화를 위해 빈도수를 그대로 사용하지만, 
    # 다른 가중치 할당 방식을 사용할 수도 있습니다.
    weights = top_words / top_words.max()  # 최대 빈도수로 정규화하여 가중치를 계산합니다.
    return weights.to_dict()

# 사용자별 가장 흔한 단어의 가중치를 계산합니다.
top_words_weights = get_top_words_weights(music_data['Processed_Lyrics'])

# 사용자의 가사 프로필을 만들 때, 가장 흔한 단어에 가중치를 주어 벡터를 계산하는 함수를 수정합니다.
def create_weighted_lyrics_profile(lyrics_list, w2v_model, top_words_weights):
    lyrics_vectors = []
    for lyrics in lyrics_list:
        # lyrics 벡터의 평균을 계산하기 전에 각 단어에 대한 가중치를 고려합니다.
        weighted_vectors = []
        for word in lyrics:
            if word in w2v_model.wv:  # 모델의 단어장에 있는 경우에만 처리합니다.
                weight = top_words_weights.get(word, 1)  # 단어에 대한 가중치를 가져옵니다.
                weighted_vectors.append(w2v_model.wv[word] * weight)
        if weighted_vectors:  # 가중치가 적용된 벡터의 평균을 계산합니다.
            lyrics_vectors.append(np.mean(weighted_vectors, axis=0))
    return np.mean(lyrics_vectors, axis=0) if lyrics_vectors else np.zeros(w2v_model.vector_size)

# 사용자별 프로필 벡터를 생성합니다.
user_id = 'UGHUjwj5'
user_lyrics = music_data[music_data['USER_ID'] == user_id]['Processed_Lyrics']
user_profile_vector = create_weighted_lyrics_profile(user_lyrics, w2v_model, top_words_weights)

In [15]:
# 특정 사용자 ID에 대한 사용자의 청취 기록을 필터링'TOt3FMdc'
user_specific_log = filtered_music_log_df[filtered_music_log_df['USER_ID'] == user_id]

# 특정 사용자의 장르별 플레이 횟수를 계산
user_specific_genre_counts = user_specific_log['GENRE'].value_counts()

# 특정 사용자의 상위 3개 장르를 가져옵니다.
user_specific_top_genres = user_specific_genre_counts.head(5).index.tolist()

# 사용자 상위 장르와 일치하는 노래에 대해 music_total_with_genre 데이터 프레임 필터링
user_specific_top_genres_songs_df = music_tag_data[music_tag_data['Genre'].isin(user_specific_top_genres)]

In [16]:
# 태그 데이터를 전처리하는 함수를 정의합니다.
def preprocess_tags(tag_string):
    # '#' 기호를 기준으로 태그를 분리합니다.
    tags = tag_string.strip().split('#')
    # 빈 문자열을 제거합니다.
    tags = [tag for tag in tags if tag]  # 공백 태그 제거
    return tags

# 태그 데이터에 전처리 함수를 적용합니다.
user_specific_top_genres_songs_df['Processed_Tags'] = user_specific_top_genres_songs_df['Tag'].apply(preprocess_tags)

/tmp/ipykernel_734/1085869089.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_specific_top_genres_songs_df['Processed_Tags'] = user_specific_top_genres_songs_df['Tag'].apply(preprocess_tags)


In [17]:
# 태그를 벡터로 변환하는 함수를 정의합니다.
def vectorize_tags(tags, w2v_model):
    tag_vectors = []
    for tag in tags:
        # 태그 내의 각 단어에 대해 벡터를 얻고 평균을 계산합니다.
        tag_word_vectors = [w2v_model.wv[word] for word in tag.split() if word in w2v_model.wv]
        if tag_word_vectors:  # 태그가 모델 단어장에 있는 경우에만 평균 벡터를 계산합니다.
            tag_vectors.append(np.mean(tag_word_vectors, axis=0))
    return np.mean(tag_vectors, axis=0) if tag_vectors else np.zeros(w2v_model.vector_size)

# 각 태그를 벡터로 변환합니다.
user_specific_top_genres_songs_df['Tag_Vector'] = user_specific_top_genres_songs_df['Processed_Tags'].apply(lambda tags: vectorize_tags(tags, w2v_model))

/tmp/ipykernel_734/224402154.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_specific_top_genres_songs_df['Tag_Vector'] = user_specific_top_genres_songs_df['Processed_Tags'].apply(lambda tags: vectorize_tags(tags, w2v_model))


In [18]:
# 사용자 프로필 벡터와 모든 태그 벡터 사이의 코사인 유사도를 계산하고 상위 N개의 추천과 함께 유사도를 반환하는 함수
def recommend_songs_with_similarity(user_profile_vector, tag_vectors, songs_data, top_n=20):
    # 사용자 프로필 벡터를 코사인 유사도 계산을 위해 reshape
    user_vector_reshaped = user_profile_vector.reshape(1, -1)

    # 모든 태그 벡터와의 유사도 계산
    similarity_scores = cosine_similarity(user_vector_reshaped, tag_vectors)[0]

    # 유사도 점수를 기반으로 상위 N개의 인덱스를 가져옵니다
    top_indices = similarity_scores.argsort()[-top_n:][::-1]

    # 상위 N개의 노래 추천 정보와 유사도 점수를 함께 반환
    recommendations_with_scores = songs_data.iloc[top_indices]
    recommendations_with_scores['similarity'] = similarity_scores[top_indices]
    return recommendations_with_scores[['Title', 'Artist', 'Tag', 'similarity']]

# 모든 태그 벡터를 하나의 배열로 추출합니다.
tag_vectors_matrix = np.array(list(user_specific_top_genres_songs_df['Tag_Vector']))

# 'UGqP66iE' 사용자 ID에 대한 노래 추천을 받고 유사도 점수를 포함하여 출력합니다.
# user_profile_vector_for_similarity = user_profiles[user_id_to_recommend]  # 해당 사용자의 프로필 벡터를 가져옵니다.
recommendations_with_similarity = recommend_songs_with_similarity(user_profile_vector, tag_vectors_matrix, user_specific_top_genres_songs_df)
recommendations_with_similarity

/tmp/ipykernel_734/736783372.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommendations_with_scores['similarity'] = similarity_scores[top_indices]


,Title,Artist,Tag,similarity
53973,사랑한번 눈물나게,오현란,#발라드한#애절한#외로울때,0.672603
60281,죄인,이은미,#발라드한#애절한#외로울때,0.672603
59278,일산호수공원 (duet with 이루마),더필름(The Film),#산책/여행#외로울때#출근/퇴근길,0.638814
47750,너라서 가능했던,슈가볼(Sugarbowl),#감성적인#국내#밤/새벽#외로울때#이별/슬픔,0.638814
48457,놀,박화요비,#감성적인#밤/새벽#외로울때#이별/슬픔,0.638814
48608,눈물 뒤에 숨다,몽실이 시스터즈,#산책/여행#외로울때#출근/퇴근길,0.638814
46745,나의 여름,이예린,#밤/새벽#외로울때,0.638814
54577,선물,유발이의 소풍,#산책/여행#외로울때#출근/퇴근길,0.638814
48350,네가 불던 날,성시경,#산책/여행#외로울때#출근/퇴근길,0.638814
47944,너에게 나는 아무것도 아닐 것 같아,경제환,#감성적인#국내#밤/새벽#외로울때#이별/슬픔,0.638814


In [22]:
from sklearn.metrics.pairwise import euclidean_distances

def recommend_songs_with_euclidean(user_profile_vector, tag_vectors, songs_data, top_n=20):
    # 사용자 프로필 벡터를 유클리드 거리 계산을 위해 reshape
    user_vector_reshaped = user_profile_vector.reshape(1, -1)

    # 모든 태그 벡터와의 유클리드 거리 계산
    distances = euclidean_distances(user_vector_reshaped, tag_vectors)[0]

    # 유클리드 거리를 유사도 점수로 변환 (거리가 가까울수록 유사도가 높게 나타남)
    similarity_scores = 1 / (1 + distances)

    # 유사도 점수를 기반으로 상위 N개의 인덱스를 가져옵니다
    top_indices = similarity_scores.argsort()[-top_n:][::-1]

    # 상위 N개의 노래 추천 정보와 유사도 점수를 함께 반환
    recommendations_with_scores = songs_data.iloc[top_indices]
    recommendations_with_scores['similarity'] = similarity_scores[top_indices]
    return recommendations_with_scores[['Title', 'Artist', 'Tag', 'similarity']]

# 사용 예제
# 'UGqP66iE' 사용자 ID에 대한 노래 추천을 받고 유사도 점수를 포함하여 출력합니다.
recommendations_with_euclidean = recommend_songs_with_euclidean(user_profile_vector, tag_vectors_matrix, user_specific_top_genres_songs_df)
recommendations_with_euclidean


/tmp/ipykernel_734/804633007.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommendations_with_scores['similarity'] = similarity_scores[top_indices]


,Title,Artist,Tag,similarity
54833,소란스러운 날,김보경 (NEON),#국내#목소리/음색#발라드한#애절한,0.325600
43441,가는 세월,서유석,#1970년대#7080#7080세대#국내#발라드한#애절한#어쿠스틱한#올디스#트로트,0.325600
62966,화;월 (花月),우예린,#OST#감성적인#국내#애절한,0.325600
62875,혼자 밥 먹지마,황치열,#국내#목소리/음색#발라드한#애절한,0.325600
57019,여러분,윤복희,#1970년대#7080#7080세대#발라드한#애절한#올디스#트로트,0.325600
57206,연(緣),포레스텔라(Forestella),#OST#감성적인#국내#애절한,0.325600
62834,호랑수월가,탑현,#OST#감성적인#국내#애절한,0.325600
50016,떨어진 잎새,전영록,#1970년대#7080#7080세대#발라드한#애절한#올디스#트로트,0.325600
58407,운명,백지영,#국내#목소리/음색#발라드한#애절한,0.325600
49212,달무리,송소희,#OST#감성적인#국내#애절한,0.325600
